In [59]:
from mip import *



In [60]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

In [88]:
instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

In [89]:
instance = instance1

In [85]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])

model = Model()

max_distance = np.max(instance["distances"], axis=1).sum()

# x[i, j, k] is 1 if the courier i is at the node j at time k
x = model.add_var_tensor((m, n + 1, n + 2), var_type=BINARY, name="x")
# model += x <= n + 1
# model += x >= 1

# y[i] is the distance traveled by the courier i
y = model.add_var_tensor((m,), var_type=INTEGER, name="y")
model += y >= 0
model += y <= max_distance

# z[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
z = model.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="z")

# t[i, j] is 1 if the courier i carries the package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

In [64]:
# couriers start and end at the depot (n + 1)

model += x[:, 0] == n + 1
model += x[:, n] == n + 1

# t[i, j] is 1 if the courier i carries the package j
for i in range(m):
    for j in range(n):
        model += np.sum(x[i, j, :]) == t[i, j]

# each courier does not exceed its capacity
for i in range(m):
    model += np.sum(t[i, :] * s) <= l[i]

# each package is carried by exactly one courier
for j in range(n):
    model += np.sum(t[:, j]) == 1

# z[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            for k in range(n + 1):
                z[i, j1, j2] <= x[i, j1, k]
                z[i, j1, j2] <= x[i, j2, k + 1]
                z[i, j1, j2] >= x[i, j1, k] + x[i, j2, k + 1] - 1

model.objective = minimize(np.max(y))

AttributeError: 'Var' object has no attribute 'expr'

In [100]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])

# deposit = n + 1

# We have n + 1 nodes in total

model2 = Model()

# x[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
x = model2.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="x")

# y[i] is the distance traveled by the courier i
y = model2.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i, j] is 1 if the courier i carries the package j
t = model2.add_var_tensor((m, n), var_type=BINARY, name="t")

# o[i] is 1 if if the courier goes outside
o = model2.add_var_tensor((m,), var_type=BINARY, name="o")


"""# All couriers start at the starting deposit
for i in range(m):
    model2 += np.sum(x[i, n + 1, :]) == 1
    model2 += np.sum(x[i, :, n + 1]) == 0

# All couriers end at the arrival deposit
for i in range(m):
    model2 += np.sum(x[i, :, n + 2]) == 1
    model2 += np.sum(x[i, n + 2, :]) == 0"""

# If the courier stays home, he will not go anywhere
for i in range(m):
    # Blocks travels between packages
    model2 += x[i, :n, :n] <= o[i]
    # Blocks travels from deposit to packages
    # TODO: Check if it should be
    # model2 += np.sum(x[i, n, :n]) == 1
    # same for the other one
    model2 += x[i, n, :n] <= o[i]
    # (Opt.) Blocks travels from packages to deposit
    model2 += x[i, :n, n] <= o[i]

# If the courier goes outside, it cannot do deposit->deposit
for i in range(m):
    model2 += x[i, n, n] <= 1 - o[i]

for i in range(m):
    model2 += np.sum(x[i, n, :]) == 1
    model2 += np.sum(x[i, :, n]) == 1


# t[i, j] is 1 if the courier i carries the package j
for i in range(m):
    for j in range(n):
        model2 += np.sum(x[i, j, :]) == t[i, j]

# All packages are carried by exactly one courier
for j in range(n):
    model2 += np.sum(t[:, j]) == 1

# If a courier arrives to a package, it has to leave from that package
for i in range(m):
    for j in range(n):
        courier_arrives = np.sum(x[i, :, j])
        courier_leaves = np.sum(x[i, j, :])
        # (not arrives) or leaves
        model2 += (1 - courier_arrives) + courier_leaves >= 1


"""# Each node must be a middle stop exactly once
for j in range(n):
    model2 += np.sum(x[i, j, :]) == 1
    model2 += np.sum(x[i, :, j]) == 1"""

# Each courier does not exceed its capacity
for i in range(m):
    model2 += (t[i, :] @ s) <= l[i]

# y[i] is the distance traveled by the courier i
for i in range(m):
    model2 +=  np.sum(x[i, :, :] * np.array(instance["distances"])) == y[i]

# You cannot travel from a node to itself
for i in range(m):
    for j in range(n):
        model2 += x[i, j, j] == 0


z = model2.add_var(name="z", var_type=INTEGER, lb=0, ub=max_distance)
model2 += np.max(y) == z

model2.objective = minimize(z)

model2.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [101]:
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            if x[i, j1, j2].x == 1:
                print(f"Courier {i} travels from {j1} to {j2}")

Courier 0 travels from 1 to 3
Courier 0 travels from 3 to 4
Courier 0 travels from 4 to 7
Courier 0 travels from 7 to 1
Courier 1 travels from 2 to 7
Courier 1 travels from 5 to 2
Courier 1 travels from 7 to 5
Courier 2 travels from 0 to 6
Courier 2 travels from 6 to 7
Courier 2 travels from 7 to 0


In [103]:
print(model2.objective_value)

12.0


In [ ]:
print([(i, j) for j in range(n) for i in range(m) if t[i, j].x == 1])

[(0, 0), (0, 1)]
